Instalação das bibliotecas:


In [ ]:
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
!pip install llama-index ipywidgets
!pip install pypdf transformers einops accelerate langchain bitsandbytes sentence-transformers
!pip install gradio
!pip install deep-translator

In [ ]:
!pip install --upgrade llama-index-llms-huggingface

Login na Hugging Face:

*Necessário para acessar os modelos de linguagem hospedados no Hugging Face.*

In [ ]:
!huggingface-cli login

Importação de bibliotecas:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gradio as gr
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

LLAMA2_13B_CHAT é o nome do modelo que será utilizado. O system_prompt define as instruções que guiarão o comportamento do assistente virtual, especificando que ele deve responder em português e fornecer explicações detalhadas.

In [ ]:
# Configurar o modelo e tokenizer
LLAMA2_13B_CHAT = "meta-llama/Llama-2-13b-chat-hf"
system_prompt = """
Você é um assistente virtual especializado em ajudar estudantes com perguntas sobre diversos tópicos acadêmicos, incluindo negócios, tecnologia, biologia, ciências, e matemática. Responda SEMPRE em português, independentemente da pergunta ou do contexto, de forma clara e concisa. Forneça explicações detalhadas e exemplos práticos, se necessário.

Se uma pergunta não fizer sentido ou não for factualmente coerente, explique o porquê de forma educada, ao invés de responder algo incorreto. Se você não souber a resposta para uma pergunta, não compartilhe informações falsas.

Para perguntas acadêmicas, forneça explicações completas. Não forneça recomendações de coisas que não existem.

Seja sempre útil, respeitoso, e educado em suas respostas. Caso um aluno seja desrespeitoso, responda de forma educada.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + system_prompt + "<</SYS>>\n\n{query_str}[/INST] "
)

# Carregar o modelo e tokenizer
tokenizer = AutoTokenizer.from_pretrained(LLAMA2_13B_CHAT)
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(
    LLAMA2_13B_CHAT,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Configurar LLM
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=500,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=LLAMA2_13B_CHAT,
    model_name=LLAMA2_13B_CHAT,
    model=model
)

In [ ]:
Settings.llm = llm  # Configurar o LLM
Settings.embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")  # Configurar o modelo de embeddings

# Carregar documentos
documents = SimpleDirectoryReader("materiais").load_data()

# Criar o índice
index = VectorStoreIndex.from_documents(documents)

# Criar mecanismo de consulta
query_engine = index.as_query_engine()

In [ ]:
from langdetect import detect
from deep_translator import GoogleTranslator

# Instanciar o tradutor
translator = GoogleTranslator(source='auto', target='pt')

# Função para verificar e traduzir para português
def translate_to_portuguese(text):
    detected_lang = detect(text)
    if detected_lang != 'pt':
        # Traduzir para o português
        translated = translator.translate(text)
        return translated
    return text

Interface Gradio:

In [ ]:
# Função para consultar o `query_engine` e gerar uma resposta
def chatbot_response(message, history):
    response = query_engine.query(message)

    # Garantir que a resposta esteja em português
    response_in_portuguese = translate_to_portuguese(str(response))
    return response_in_portuguese

# Criar a interface Gradio no estilo de um chatbot
chatbot = gr.ChatInterface(fn=chatbot_response,
                           title="Assistente Virtual",
                           description="Converse com o chatbot e receba respostas para suas perguntas!",
                           theme="compact")

# Iniciar a interface Gradio com compartilhamento
if __name__ == "__main__":
    chatbot.launch(share=True)